In [ ]:
import pandas as pd #for data manipulation and analysis, used for importing the data file
import numpy as np #importing numpy library for numerical computations.
from matplotlib import pyplot as plt #importing pyplot module from matplotlib for data visualization.
from mpl_toolkits.mplot3d import Axes3D #provides a set of tools for creating 3D plots using matplotlib
import seaborn as sns #importing seaborn library for advanced data visualization.
import math #provides mathemathemical functions and constants.
from sklearn.tree import DecisionTreeClassifier #importing DecisionTreeClassifier algorithm from scikit-learn for building decision trees.
from sklearn import tree #importing tree module from scikit-learn for visualizing decision trees.
from sklearn import svm #importing svm module from scikit-learn for Support Vector Machine algorithm.
from sklearn.linear_model import Perceptron # importing Perceptron algorithm from scikit-learn for building Perceptron models.
from sklearn.model_selection import cross_val_score #importing cross_val_score method from scikit-learn for cross-validation.
from sklearn.metrics import confusion_matrix #importing confusion_matrix metric from scikit-learn for evaluating classification performance.
import tkinter as tk
from tkinter import ttk


parkinson = pd.read_csv("parkinsons.csv") #importing the data file from the same folder
X = parkinson[['PPE','MDVP:Fhi(Hz)','Shimmer:APQ5','RPDE', 'spread2', 'MDVP:Fo(Hz)', 'D2', 'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'MDVP:Flo(Hz)']] # X are the features or attributes
Y = parkinson['status'] #Sets label as 0 or 1 repectively
X = X.copy()

upper_limit = X['MDVP:Fhi(Hz)'].quantile(0.90) #Calculates the 90th percentile and stores it in the variable upper_limit
lower_limit = X['MDVP:Fhi(Hz)'].quantile(0.10) #Calculates the 10th percentile and stores it in the variable lower_limit
X['MDVP:Fhi(Hz)'] = np.where(X['MDVP:Fhi(Hz)']>= upper_limit, #if the value in the 'MDVP:Fhi(Hz)' column is greater than or equal to the 'upper_limit' value we found earlier, then we replace it with the 'upper_limit' value.
        upper_limit, #Replace the value in the 'MDVP:Fhi(Hz)' column with if it meets the condition in the previous line.
        np.where(X['MDVP:Fhi(Hz)'] <= lower_limit, #we are checking if the value in the 'MDVP:Fhi(Hz)' column is less than or equal to the 'lower_limit' value we found earlier.
        lower_limit, #Replaces the value with lower limit.
        X['MDVP:Fhi(Hz)'])) #The outliers corresponding to the upper limit are exchanged with the value of upper limit and the values corresponding to lower limit are assigned that. Values falling in range are kept unchanged.


upper_limit = X['MDVP:Flo(Hz)'].quantile(0.95) #Calculates the 95th percentile and stores it in upper_limit
lower_limit = X['MDVP:Flo(Hz)'].quantile(0.05) #Calculates the 5th percentile and stores it in lower_limit
X['MDVP:Flo(Hz)'] = np.where(X['MDVP:Flo(Hz)']>= upper_limit, 
        upper_limit,
        np.where(X['MDVP:Flo(Hz)'] <= lower_limit,
        lower_limit,
        X['MDVP:Flo(Hz)'])) #The outliers corresponding to the upper limit are exchanged with the value of upper limit and the values corresponding to lower limit are assigned that. Values falling in range are kept unchanged.

upper_limit = X['MDVP:Shimmer'].quantile(0.95)
lower_limit = X['MDVP:Shimmer'].quantile(0.05)
X['MDVP:Shimmer'] = np.where(X['MDVP:Shimmer']>= upper_limit,
        upper_limit,
        np.where(X['MDVP:Shimmer'] <= lower_limit,
        lower_limit,
        X['MDVP:Shimmer']))

upper_limit = X['MDVP:Shimmer(dB)'].quantile(0.90)
lower_limit = X['MDVP:Shimmer(dB)'].quantile(0.10)
X['MDVP:Shimmer(dB)'] = np.where(X['MDVP:Shimmer(dB)']>= upper_limit,
        upper_limit,
        np.where(X['MDVP:Shimmer(dB)'] <= lower_limit,
        lower_limit,
        X['MDVP:Shimmer(dB)']))

upper_limit = X['Shimmer:APQ5'].quantile(0.90)
lower_limit = X['Shimmer:APQ5'].quantile(0.10)
X['Shimmer:APQ5'] = np.where(X['Shimmer:APQ5']>= upper_limit,
        upper_limit,
        np.where(X['Shimmer:APQ5'] <= lower_limit,
        lower_limit,
        X['Shimmer:APQ5']))

upper_limit = X['spread2'].quantile(0.95)
lower_limit = X['spread2'].quantile(0.05)
X['spread2'] = np.where(X['spread2']>= upper_limit,
        upper_limit,
        np.where(X['spread2'] <= lower_limit,
        lower_limit,
        X['spread2']))

upper_limit = X['D2'].quantile(0.95)
lower_limit = X['D2'].quantile(0.10)
X['D2'] = np.where(X['D2']>= upper_limit,
        upper_limit,
        np.where(X['D2'] <= lower_limit,
        lower_limit,
        X['D2']))


upper_limit = X['PPE'].quantile(0.95)
lower_limit = X['PPE'].quantile(0.05)
X['PPE'] = np.where(X['PPE']>= upper_limit,
        upper_limit,
        np.where(X['PPE'] <= lower_limit,
        lower_limit,
        X['PPE']))

def MinMaxScale(X): #This line defines a function called MinMaxScale that takes a single argument X.
    # Get the column names
    col_names = X.columns # This line gets the column names of the input X and assigns them to a variable called col_names

    # Scale each column
    for col in col_names: #This line starts a loop that goes through each column in col_names one at a time and assigns the current column to a variable called col.
        col_min = X[col].min() # gets the minimum value in the current column col of the input X and assigns it to a variable called col_min.
        col_max = X[col].max() # gets the maximum value in the current column col of the input X and assigns it to a variable called col_max.
        X[col] = (X[col] - col_min) / (col_max - col_min) #scales the current column col of the input X by subtracting the minimum value of the column and then dividing by the range of the column (i.e., the difference between the maximum and minimum values).

    return X # returns the scaled input X after all the columns have been scaled.


Standardized = MinMaxScale(X) #this code is scaling the values of each column of the dataset to fit between the range of 0 to 1. 

#defines a function called train_test_spilt that takes four parameters: X (the input features), Y (the target variable), test_size (the proportion of the data to use for testing), and random_state (an optional parameter to set a random seed for reproducibility)
def train_test_spilt(X,Y,test_size,random_state=None): 
    X=X.sample(frac=1) #shuffles the rows of the input feature matrix X so that they are in a random order.
    n_train = math.floor(test_size * X.shape[0]) #calculates the number of rows to use for training based on the test_size parameter and the number of rows in the input feature matrix X.
    X_train = X[n_train:] #Selects the rows of the input feature matrix X starting from index n_train to use for training.
    Y_train = Y[n_train:] #selects the corresponding rows of the target variable Y to use for training.
    X_test = X[:n_train] #selects the rows of the input feature matrix X up to index n_train to use for testing.
    Y_test = Y[:n_train] #selects the corresponding rows of the target variable Y to use for testing.
    return X_train, Y_train, X_test,Y_test #our objects: the training input feature matrix X_train, the training target variable Y_train, the testing input feature matrix X_test, and the testing target variable Y_test

X_train, Y_train, X_test, Y_test = train_test_spilt(X, Y, test_size=0.2, random_state=2) #This code randomly splits the dataset X and target variable Y into training and testing sets with a 80:20 ratio, using a random state of 2, and assigns them to X_train, Y_train, X_test, and Y_test.


svm_linear = svm.SVC(kernel='linear',C=100) #Createing a learning model called "svm_linear". This model uses a type of algorithm called "SVC".
svm_linear.fit(X_train, Y_train) #Model is trained on some data using the ".fit()" method. The training data is split into two parts: the input data ("X_train") and the output data ("Y_train"). The model is trained to learn the relationship between the input data and the output data, so that it can predict the output values for new input data that it hasn't seen before.

svm_polynomial = svm.SVC(kernel='poly',degree=3,C=100) #creates an instance of the support vector machine (SVM) classifier with a polynomial kernel function. The degree of the polynomial kernel is set to 3.
svm_polynomial.fit(X_train, Y_train) ##Model is trained on some data using the ".fit()" method. The training data is split into two parts: the input data ("X_train") and the output data ("Y_train"). The model is trained to learn the relationship between the input data and the output data, so that it can predict the output values for new input data that it hasn't seen before.

svm_rbf = svm.SVC(kernel='rbf',gamma=0.7,C=100) #Sets the kernel parameter to 'rbf' stands for radial basis function. It also sets the gamma parameter to 0.7, which is used to control the shape of the decision boundary.
svm_rbf.fit(X_train, Y_train)#Model is trained on some data using the ".fit()" method. The training data is split into two parts: the input data ("X_train") and the output data ("Y_train"). The model is trained to learn the relationship between the input data and the output data, so that it can predict the output values for new input data that it hasn't seen before.

perceptron = Perceptron() # Creates a new Perceptron object that can be used to train and predict using a perceptron classifier.
perceptron.fit(X_train, Y_train)#Model is trained on some data using the ".fit()" method. The training data is split into two parts: the input data ("X_train") and the output data ("Y_train"). The model is trained to learn the relationship between the input data and the output data, so that it can predict the output values for new input data that it hasn't seen before.

decison_tree = DecisionTreeClassifier() #creates an instance of the decision tree classifier, which will be used to train a decision tree model on our dataset.
decison_tree.fit(X_train, Y_train) ##Model is trained on some data using the ".fit()" method. The training data is split into two parts: the input data ("X_train") and the output data ("Y_train"). The model is trained to learn the relationship between the input data and the output data, so that it can predict the output values for new input data that it hasn't seen before.




class PlaceholderEntry(tk.Entry):
    def __init__(self, master=None, placeholder="", color='grey', *args, **kwargs):
        super().__init__(master, *args, **kwargs)
        self.placeholder = placeholder
        self.placeholder_color = color
        self.default_fg_color = self['fg']

        self.bind("<FocusIn>", self.on_focus_in)
        self.bind("<FocusOut>", self.on_focus_out)
        self.on_focus_out(None)

    def on_focus_in(self, event):
        if self.get() == self.placeholder:
            self.delete(0, 'end')
            self.config(fg=self.default_fg_color)

    def on_focus_out(self, event):
        if not self.get():
            self.insert(0, self.placeholder)
            self.config(fg=self.placeholder_color)



def predict():
    input_data = (
        float(feature1_entry.get()),
        float(feature2_entry.get()),
        float(feature3_entry.get()),
        float(feature4_entry.get()),
        float(feature5_entry.get()),
        float(feature6_entry.get()),
        float(feature7_entry.get()),
        float(feature8_entry.get()),
        float(feature9_entry.get()),
        float(feature10_entry.get())
    )


    pd_input = pd.DataFrame([input_data], columns=X.columns)
    input_data_as_numpy_array = pd_input.to_numpy()
    input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)
    def Standardizings(X): #The function takes one parameter, which is X.
        mean = X.mean() #This calculates the mean of the input data, which is stored in a variable called mean.
        std = X.std() #This calculates the standard deviation of the input data, which is stored in a variable called std.
        standardized_row = (X - mean) / std #This is the formula for standardization. It subtracts the mean from each data point and divides it by the standard deviation to obtain a standardized value for each data point. The standardized values are stored in a variable called standardized_row.
        return standardized_row #This is the output of the function. It returns the standardized values of the input data.

    Standardize = Standardizings(input_data_reshaped) #The function will then standardize the data and return the standardized row as the output. The result is then assigned to a variable named Standardize.
    stand = pd.DataFrame(Standardize, columns=X.columns) #creates a new DataFrame called pd_input with a single row of input data, where the column names of the new DataFrame match those of an existing DataFrame X.
    classifier_type = classifier_var.get()  # Get the selected kernel type
    if classifier_type == "SVM Linear":
        svm_model = svm_linear
    elif classifier_type == "SVM Polynomial":
        svm_model = svm_polynomial
    elif classifier_type == "SVM RBF":
        svm_model = svm_rbf
    elif classifier_type == "Perceptron":
        svm_model = perceptron        
    elif classifier_type == "Decison Tree":
        svm_model = decison_tree
        
    prediction = svm_model.predict(stand)
#     prediction = svm_linear.predict(stand) #Using the svm_polynomial model we trained earlier to make a prediction on the Standardize input data, which has been standardized and reshaped into a numpy array. This prediction will be stored in the prediction variable.

    if prediction[0] == 0:
        status_label.config(text="The Person does not have Parkinson's Disease")
    else:
        status_label.config(text="The Person has Parkinson's Disease")

# Create the tkinter window
root = tk.Tk()
root.geometry('450x550')
root.minsize(450, 550)
root.maxsize(450, 550)
root.config(bg='#E7D0DF')
root.wm_title("Parkinson's Disease Predictor")

heading = tk.Label(root, text="Parkinson's Disease Predictor", fg='#633E54', bg='#E7D0DF', font=('Times New Roman', 26))

classifier_label = tk.Label(root, text='Classifier Type', fg='black', bg='#E7D0DF', font=('Times New Roman', 13))
classifier_var = tk.StringVar()
classifier_dropdown = ttk.OptionMenu(root, classifier_var, "SVM Linear", "SVM Linear", "SVM Polynomial", "SVM RBF", "Perceptron", "Decison Tree")
classifier_dropdown.config(width=27)

feature1_label = tk.Label(root, text='PPE', fg='black', bg='#E7D0DF', font=('Times New Roman', 13))
feature1_entry = PlaceholderEntry(root, placeholder='Range(0.044539 - 0.527367)', width=30, justify='center')

feature2_label = tk.Label(root, text='MDVP:Fhi(Hz)', fg='black', bg='#E7D0DF', font=('Times New Roman', 13))
feature2_entry = PlaceholderEntry(root, placeholder='Range(102.145000 - 592.030000)', width=30, justify='center')

feature3_label = tk.Label(root, text='Shimmer:APQ5', fg='black', bg='#E7D0DF', font=('Times New Roman', 13))
feature3_entry = PlaceholderEntry(root, placeholder='Range(0.005700 - 0.079400)', width=30, justify='center')

feature4_label = tk.Label(root, text='RPDE', fg='black', bg='#E7D0DF', font=('Times New Roman', 13))
feature4_entry = PlaceholderEntry(root, placeholder='Range(0.256570 - 0.685151)', width=30, justify='center')

feature5_label = tk.Label(root, text='Spread2', fg='black', bg='#E7D0DF', font=('Times New Roman', 13))
feature5_entry = PlaceholderEntry(root, placeholder='Range(0.006274 - 0.450493)', width=30, justify='center')

feature6_label = tk.Label(root, text='MDVP:Fo(Hz)', fg='black', bg='#E7D0DF', font=('Times New Roman', 13))
feature6_entry = PlaceholderEntry(root, placeholder='Range(88.333000 - 260.105000)', width=30, justify='center')

feature7_label = tk.Label(root, text='D2', fg='black', bg='#E7D0DF', font=('Times New Roman', 13))
feature7_entry = PlaceholderEntry(root, placeholder='Range(1.423287 - 3.671155)', width=30, justify='center')

feature8_label = tk.Label(root, text='MDVP:Shimmer', fg='black', bg='#E7D0DF', font=('Times New Roman', 13))
feature8_entry = PlaceholderEntry(root, placeholder='Range(0.009540 - 0.119080)', width=30, justify='center')

feature9_label = tk.Label(root, text='MDVP:Shimmer(dB)', fg='black', bg='#E7D0DF', font=('Times New Roman', 13))
feature9_entry = PlaceholderEntry(root, placeholder='Range(0.085000 - 1.302000)', width=30, justify='center')

feature10_label = tk.Label(root, text='MDVP:Flo(Hz)', fg='black', bg='#E7D0DF', font=('Times New Roman', 13))
feature10_entry = PlaceholderEntry(root, placeholder='Range(65.476000 - 239.170000)', width=30, justify='center')

predict_button = tk.Button(root, text='PREDICT', fg='white', bg='black', font=('Times New Roman', 13), command=predict)
status_label = tk.Label(root, text='', fg='black', bg='#E7D0DF', font=('Times New Roman', 14), wraplength=300)

heading.grid(row=0, column=0, columnspan=2, sticky="ew", pady=10)

pady_value = 5
classifier_label.grid (row=1, column=0, sticky="e", pady=(0, pady_value))
classifier_dropdown.grid (row=1, column=1, pady=(0, pady_value))
feature1_label.grid(row=2, column=0, sticky="e", pady=(0, pady_value))
feature1_entry.grid(row=2, column=1, pady=(0, pady_value))

feature2_label.grid(row=3, column=0, sticky="e", pady=pady_value)
feature2_entry.grid(row=3, column=1, pady=pady_value)

feature3_label.grid(row=4, column=0, sticky="e", pady=pady_value)
feature3_entry.grid(row=4, column=1, pady=pady_value)

feature4_label.grid(row=5, column=0, sticky="e", pady=pady_value)
feature4_entry.grid(row=5, column=1, pady=pady_value)

feature5_label.grid(row=6, column=0, sticky="e", pady=pady_value)
feature5_entry.grid(row=6, column=1, pady=pady_value)

feature6_label.grid(row=7, column=0, sticky="e", pady=pady_value)
feature6_entry.grid(row=7, column=1, pady=pady_value)

feature7_label.grid(row=8, column=0, sticky="e", pady=pady_value)
feature7_entry.grid(row=8, column=1, pady=pady_value)

feature8_label.grid(row=9, column=0, sticky="e", pady=pady_value)
feature8_entry.grid(row=9, column=1, pady=pady_value)

feature9_label.grid(row=10, column=0, sticky="e", pady=pady_value)
feature9_entry.grid(row=10, column=1, pady=pady_value)

feature10_label.grid(row=11, column=0, sticky="e", pady=pady_value)
feature10_entry.grid(row=11, column=1, pady=pady_value)

predict_button.grid(row=12, column=0, columnspan=2, pady=10, sticky="ew", padx=10)
status_label.grid(row=13, column=0, columnspan=2, pady=10)


root.mainloop()

